In [1]:
!pip install tweetnlp


Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import json
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
filepath = './final_dataset.json'
df = pd.read_json(filepath)
df.head()

,time,timezone,day,username,tweet,replies_count,retweets_count,likes_count,hashtags_count,cashtags_count
0,13:17:12,-400,3,sheezy0,Adam Cole Praises Kevin Owens + A Preview For ...,0,0,1,0,0
1,12:47:16,-400,3,sheezy0,FOX BUSINESS SPECIAL: US oil exports soaring —...,0,0,0,0,0
2,12:47:15,-400,3,sheezy0,Feds to investigate Mass. town's rejection of ...,0,0,0,0,0
3,12:47:15,-400,3,sheezy0,Ferocious Southern California wildfire brings ...,0,0,0,0,0
4,12:47:14,-400,3,sheezy0,TODD STARNES: Here's one school that does the ...,0,0,0,0,0


In [4]:
from datetime import datetime, time, timedelta
import math

filepath = './final_dataset.json'
df = pd.read_json(filepath)
df.head()


def get_time_range(time_str):
    t = datetime.strptime(time_str, '%H:%M:%S').time()
    hour = t.hour
    range_num = math.ceil((hour + 1) / 4)
    if range_num == 0:
        range_num = 1
    return range_num

df["time_range"]=df["time"].apply(get_time_range)

# # # Drop unnecessary columns
df = df.drop(["time", "day", "timezone"], axis=1)

# # # Convert time of day categories to one-hot encoded features
df = pd.concat([df, pd.get_dummies(df["time_range"])], axis=1)
df = df.drop(["time_range"], axis=1)

import numpy as np

df['likes_count'] = np.log(df['likes_count'] + 1)
df['retweets_count'] = np.log(df['retweets_count'] + 1)
df['replies_count'] = np.log(df['replies_count'] + 1)

df['engagement_rate'] = 0.5*df['likes_count'] + df['replies_count'] + df['retweets_count']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# # Normalize the 'sum' column to range [0, 1]
df['engagement_rate_normalized'] = scaler.fit_transform(df[['engagement_rate']])

hashtag_df = df[['hashtags_count']]
cashtag_df = df[['cashtags_count']]

hashtag_df['hashtag_count_normalized'] = scaler.fit_transform(hashtag_df)
cashtag_df['cashtag_count_normalized'] = scaler.fit_transform(cashtag_df)

df['hashtags_count'] = hashtag_df['hashtag_count_normalized']
df['cashtags_count'] = cashtag_df['cashtag_count_normalized']

final_data = df.drop(['replies_count', 'retweets_count', 'likes_count', 'engagement_rate'], axis=1)
# final_data = final_data.drop(['username'], axis=1)
final_data.columns = final_data.columns.astype(str)
final_data.head(200)

/tmp/ipykernel_130325/3898742165.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hashtag_df['hashtag_count_normalized'] = scaler.fit_transform(hashtag_df)
/tmp/ipykernel_130325/3898742165.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cashtag_df['cashtag_count_normalized'] = scaler.fit_transform(cashtag_df)


,username,tweet,hashtags_count,cashtags_count,1,2,3,4,5,6,engagement_rate_normalized
0,sheezy0,Adam Cole Praises Kevin Owens + A Preview For ...,0.000000,0.0,0,0,0,1,0,0,0.014647
1,sheezy0,FOX BUSINESS SPECIAL: US oil exports soaring —...,0.000000,0.0,0,0,0,1,0,0,0.000000
2,sheezy0,Feds to investigate Mass. town's rejection of ...,0.000000,0.0,0,0,0,1,0,0,0.000000
3,sheezy0,Ferocious Southern California wildfire brings ...,0.000000,0.0,0,0,0,1,0,0,0.000000
4,sheezy0,TODD STARNES: Here's one school that does the ...,0.000000,0.0,0,0,0,1,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
195,davdburnett,The Handfasting is everything a good romance s...,0.083333,0.0,0,0,1,0,0,0,0.000000
196,davdburnett,"RT @owen_author: Os Inadmissíveis é, na sua es...",0.055556,0.0,0,0,1,0,0,0,0.126606
197,davdburnett,RT @pamlecky: FOOTPRINTS IN THE SAND\nBook 2 o...,0.083333,0.0,0,0,1,0,0,0,0.152605
198,davdburnett,RT @amaraaminx: Prey or Mate? Destiny is ticki...,0.083333,0.0,0,0,1,0,0,0,0.130634


In [5]:
# final_data.describe()

# Calculate the percentage of 0 values in the specific column
zero_percentage = (final_data['engagement_rate_normalized'] == 0).sum() / len(final_data) * 100

# Determine the number of rows to remove to get the desired percentage of 0 values
remove_num = int(len(final_data) * (zero_percentage - 10) / 100)
print(remove_num)

# Randomly select the rows to remove
remove_indices = final_data[final_data['engagement_rate_normalized'] == 0].sample(n=remove_num).index

# Remove the selected rows from the dataframe
final_data = final_data.drop(remove_indices)

final_data.describe()



188083


,hashtags_count,cashtags_count,1,2,3,4,5,6,engagement_rate_normalized
count,443728.000000,443728.000000,443728.000000,443728.000000,443728.000000,443728.000000,443728.000000,443728.000000,443728.000000
mean,0.010533,0.000154,0.096117,0.107699,0.196222,0.245231,0.202437,0.152294,0.108105
std,0.034377,0.005738,0.294753,0.310000,0.397139,0.430225,0.401817,0.359306,0.129802
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014647
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046430
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.160878
max,1.000000,0.800000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
!pip install tweet-preprocessor tweet-preprocessor[tweetnlp]

from preprocessor.api import clean

final_data["tweet"] = [clean(tweet) for tweet in final_data['tweet']]
# final_data.head()
final_data.to_pickle('final_data_cleaned.pkl')


Defaulting to user installation because normal site-packages is not writeable


In [5]:
final_data = pd.read_pickle('final_data_cleaned.pkl')

import tweetnlp

model = tweetnlp.load_model('sentence_embedding')
def embed_tweet(tweet):
    return model.embedding(tweet)

final_data["tweet_embeddings"] = final_data["tweet"].apply(embed_tweet)

final_data.to_pickle('final_data_with_embeddings.pkl')



/home/cj/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /home/cj/.cache/torch/sentence_transformers/cambridgeltl_tweet-roberta-base-embeddings-v1. Creating a new one with MEAN pooling.
/home/cj/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/cj/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")

KeyboardInterrupt: 

In [2]:
import pandas as pd
final_data = pd.read_pickle('final_data_with_embeddings.pkl')
final_data = final_data.drop(["username","tweet"], axis=1)

final_data = final_data.join(final_data['tweet_embeddings'].apply(pd.Series).add_prefix('vector_field_'))
final_data.to_pickle('final_data.pkl')

In [4]:
final_data.columns

Index(['hashtags_count', 'cashtags_count', '1', '2', '3', '4', '5', '6',
       'engagement_rate_normalized', 'tweet_embeddings',
       ...
       'vector_field_758', 'vector_field_759', 'vector_field_760',
       'vector_field_761', 'vector_field_762', 'vector_field_763',
       'vector_field_764', 'vector_field_765', 'vector_field_766',
       'vector_field_767'],
      dtype='object', length=778)

In [1]:
from sklearn.decomposition import IncrementalPCA
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

final_data = pd.read_pickle('final_data.pkl')
final_data = final_data.drop(["tweet_embeddings"], axis=1)

final_data.columns
X = final_data.drop(["engagement_rate_normalized"], axis=1)
y = final_data["engagement_rate_normalized"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

del X
del y
del final_data
# pca = IncrementalPCA(n_components=50, batch_size=10)
# X_train_pca = pca.fit_transform(X_train)
# X_test_pca = pca.transform(X_test)

# pca_vectors = pca.fit_transform(final_data.to_numpy())
# input_data = pd.DataFrame(pca_vectors, columns=[f"feature_{i+1}" for i in range(50)])
# input_data.head() 

In [3]:
X_train.shape[1]

776

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf = RandomForestRegressor(n_estimators=100, random_state=42) # using 100 trees
rf.fit(X_train, y_train)

# make predictions on the test set
y_pred = rf.predict(X_test)

# evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [2]:
# Define the model

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import pickle
import numpy as np

model = Sequential()
model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
# model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.01), metrics=['accuracy'])
model.compile(optimizer='adam', loss='mse')
batch_size = 64
num_epochs = 10
num_samples = X_train.shape[0]
num_batches = int(np.ceil(num_samples / batch_size))

num_epochs = 10
for j in range(num_epochs):
    print(f'Epoch: {j}')
    for i in range(int(num_batches)):
        batch_x = X_train[i*batch_size:(i+1)*batch_size]
        batch_y = y_train[i*batch_size:(i+1)*batch_size]
        model.train_on_batch(batch_x, batch_y)
    with open(f'model_{j+1}.pkl', 'wb') as f:
        pickle.dump(model, f)

# # Train the model for n epochs
# model.fit(X_train, y_train, epochs=5)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print("Mean squared error: ", loss)

# Make predictions on new data
# X_new = np.array([[...], [...], ...])  # new data
# predictions = model.predict(X_new)

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
2774/2774 [==============================] - 2s 687us/step - loss: 0.0285
Mean squared error:  0.02854808233678341
